- Author: Benjamin Du
- Date: 2021-12-11 17:21:56
- Modified: 2022-01-18 14:34:12
- Title: Control Number of Partitions of a DataFrame in Spark
- Slug: control-number-of-partitions-of-a-dataframe-in-spark
- Category: Computer Science
- Tags: Computer Science, programming, Spark, PySpark, big data, partition, repartition, maxPartitionBytes


## Tips and Traps

1. `DataFrame.repartition` repartitions the DataFrame by **hash code** of each row. 
    If you specify a (multiple) column(s) (instead of number of partitions) 
    to the method `DataFrame.repartition`,
    then hash code of the column(s) are calculated for repartition. 
    In some situations,
    there are lots of hash conflictions 
    even if the total number of rows is small (e.g., a few thousand),
    which means that
    <span style="color:red"> partitions generated might be skewed </span>.
    and causes a few long-running tasks. 
    If this ever happens, 
    it is suggested that you manually add a column
    which helps the hashing algoirthm. 
    Notice that *an existing integer column with distinct values in the DataFrame 
    is not necessarily a good column to repartition by* 
    especially when those integers are big (e.g., u64)
    as hash code of those integers can easily conflicts. 
    It is best to add a column of random numbers 
    of a column of manually curated partition indexes
    and ask Spark to repartition based on that column.
    
2. By default, 
    Spark automatically merges several small files into one partition 
    when loading a HDFS table into a DataFrame. 
    The behavior is control by the parameter `spark.sql.files.maxPartitionBytes`.
    The default value for this option is 128M
    which means that Spark keeps reading small files into one partition of a DataFrame
    until reading another file makes the size of the partition exceedes 128M. 
    Generally speaking,
    you want to keep the default value for this setting 
    as it yields optimal performance for handling large data table. 
    However,
    if your Spark application deals with small data but is CPU intensive,
    it makes more sense to set a much smaller value for `spark.sql.files.maxPartitionBytes`
    so that there are more partitions generated and yield a higher level of parallelism.
    Of course,
    you can always repartition a DataFrame manually. 

    However, 
    there's a minor drawback if you manually set `spark.sql.files.maxPartitionBytes`
    to a much smaller value (than the default one).
    The Hadoop file system does not work well with a large number of small files
    Having Spark/PySpark applications generating huge amount of small files
    not only hurt the performance of the Hadoop file system
    but might also exceed the namespace quota limitation 
    (of the directory that your application write data to). 
    A good practice in this situation is to manually reduce the number of partitions 
    after your Spark/PySpark application finish computing.
    There are a few ways to achieve this.

    - Manually repartition the output DataFrame to reduce the number of partitions.
    - Manually coalesce the output DataFrame to reduce the number of partitions.
    
    Manually repartition the output DataFrame is easy to carry out
    but it causes a full data shuffling 
    which might be expensive.
    Manually coalescing the output DataFrame (to reduce the number of partitions)
    is less expensive
    but it has a pitfall.
    Spark optimizes the physical plan 
    and might reduce the number of partitions before computation of the DataFrame.
    This is undesirable if you want to have a large number of partitions 
    to increase parallelism when computing the DataFrame
    and reduce the number of partitions when outputing the DataFrame. 
    There are a few ways to solve this problem.

    - Checkpoint the DataFrame before coalescing.
    - Cache the DataFrame and trigger a RDD `count` action 
        (unlike checkpoint, caching itself does not trigger a RDD action) 
        before coalescing.

    Generally speaking,
    caching + triggering a RDD action has a better performance than checkpoint 
    but checkpoint is more robust (to noisy Spark cluster).
    You can also manually output the DataFrame (before coalesing),
    read it back,
    and then coalesce (to reduce the number of partitions)
    and output it.
    It is equivalent to caching to disk and then trigger a RDD action.  
    Please refer to 
    [repart_hdfs](https://github.com/dclong/dsutil/blob/dev/dsutil/hadoop/utils.py#L78)
    for such an example.
        

## References 

- [Partition and Bucketing in Spark](http://www.legendu.net/misc/blog/partition-bucketing-in-spark/)

- [Coalesce and Repartition in Spark DataFrame](http://www.legendu.net/misc/blog/spark-dataframe-coalesce-repartition/)